In [1]:
from DANE_utils import jobspec
from dummyhandler import DummyHandler
import json

def pprint(obj):
    print(json.dumps(json.loads(str(obj)), indent=4))

# Jobspec

In [2]:
job = jobspec.jobspec(source_url='http://127.0.0.1/example', source_id='ITM123', source_set='NISV',
                      tasks=jobspec.taskSequential(['DOWNLOAD', 'CV', 'ASR', 'INDEX', 'DELETE']),
                    metadata={'FOO': 'BAR'})

pprint(job)

{
    "source_url": "http://127.0.0.1/example",
    "source_id": "ITM123",
    "source_set": "NISV",
    "job_id": null,
    "tasks": {
        "taskSequential": [
            "DOWNLOAD",
            "CV",
            "ASR",
            "INDEX",
            "DELETE"
        ]
    },
    "metadata": {
        "FOO": "BAR"
    },
    "priority": 1,
    "response": {}
}


In [3]:
new_job = jobspec.jobspec.from_json(job.to_json())
pprint(new_job)

{
    "source_url": "http://127.0.0.1/example",
    "source_id": "ITM123",
    "source_set": "NISV",
    "job_id": null,
    "tasks": {
        "taskSequential": [
            "DOWNLOAD",
            "CV",
            "ASR",
            "INDEX",
            "DELETE"
        ]
    },
    "metadata": {
        "FOO": "BAR"
    },
    "priority": 1,
    "response": {}
}


## API

In [4]:
# Dummy endpoint so we can 'simulate' behaviour of workflow
dummy = DummyHandler()
new_job.set_api(dummy)

print(new_job.tasks[0].api) # instance, won't be serialised with job

TypeError: Can't instantiate abstract class DummyHandler with abstract methods callback, get_dirs

In [ ]:
# register job & all tasks
new_job.register()
pprint(new_job)

In [ ]:
# Explicitly run a specific task
# No guarantee that this task can run successfully if preceding tasks havent been run
new_job.tasks[0].run()

In [ ]:
# run next task
new_job.tasks.run()

In [ ]:
job.set_api(dummy)
job.register()

while not job.isDone():
    job.run()

# Nesting Task Containers

In [ ]:
new_tasks = jobspec.taskSequential(['DOWNLOAD', 
                            jobspec.taskParallel(['CV', 'ASR']), 
                            jobspec.taskParallel(['INDEX', 'DELETE'])])
pprint(new_tasks)

In [ ]:
newer_tasks = jobspec.taskContainer.from_json(new_tasks.to_json())
pprint(newer_tasks)

In [ ]:
newer_job = jobspec.jobspec(source_url='http://127.0.0.1/example2', source_id='ITM124', source_set='NISV',
                      tasks=newer_tasks)

In [ ]:
newer_job.set_api(dummy)
newer_job.register()

In [ ]:
newest_job = jobspec.jobspec.from_json(newer_job.to_json())
newest_job.set_api(dummy) # api isn't serialised, so need to set again

newest_job.tasks[0].run()
newest_job.tasks[1].run()